<a href="https://colab.research.google.com/github/cerenyurtlu/yetgen_mock_data_analysis/blob/main/excel_note_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gerekli kütüphanelerin içe aktarımı:


*   **Pandas**

1. Excel dosyasını okuma (pd.read_excel).
2. Veri üzerinde işlem yapma (toplam not hesaplama ve filtreleme).
3. Sonuçları yeni bir sheet olarak Excel dosyasına yazma (to_excel).
*   **files**
1. Kullanıcıdan Excel dosyasını yüklemek (files.upload).
2. İşlenmiş Excel dosyasını indirmek (files.download).
*   **io**
1. Yüklenen dosyayı byte stream olarak pandas'a okutmak (io.BytesIO(uploaded['YetGen_Mock_Data.xlsx'])).
*   **openpyxl**
1. Mevcut Excel dosyasına yeni bir sheet eklemek (pd.ExcelWriter ile openpyxl motorunu kullanarak).

In [17]:
import pandas as pd
from google.colab import files
import io
from openpyxl import load_workbook

**Açıklama:**

- from google.colab import files: Google Colab'da dosya yüklemek ve indirmek için gerekli files modülünü içe aktarır.

- files.upload():Bu fonksiyon, kullanıcıya bir dosya seçmesi için bir dosya yükleme penceresi açar. Kullanıcı dosyayı seçtiğinde, bu dosya Google Colab ortamına yüklenir.

In [18]:
from google.colab import files
uploaded = files.upload()

Saving YetGen_Mock_Data.xlsx to YetGen_Mock_Data.xlsx


**Açıklama**:

- import io: io modülü, byte stream işlemleri için kullanılır. Özellikle dosya içeriğini byte stream olarak okuyabilmek için gereklidir. Bu, yüklenen dosyayı bir pandas DataFrame olarak okumak için önemli bir adımdır.
pd.read_excel(io.BytesIO(uploaded['YetGen_Mock_Data.xlsx'])):

- uploaded['YetGen_Mock_Data.xlsx']: Daha önce yüklenmiş olan dosyanın içeriğini byte stream formatında elde eder. uploaded sözlüğü içinde dosyanın içeriği, byte stream olarak saklanır.

- io.BytesIO(...): Byte stream verisini bir BytesIO nesnesine dönüştürür. Bu nesne, pandas'ın read_excel fonksiyonunun dosya benzeri bir nesne olarak işlem yapabilmesini sağlar.

- pd.read_excel(...): BytesIO nesnesinden Excel dosyasını okur ve verileri bir pandas DataFrame olarak döndürür. Bu, Excel dosyasındaki verileri işlemek ve analiz etmek için kullanılır.

- df: Oluşturulan DataFrame, Excel dosyasındaki verilerin pandas veri yapısına dönüştürülmüş halidir. Bu veri yapısı, veriler üzerinde çeşitli işlemler yapabilmenizi sağlar (örneğin, filtreleme, toplama, analiz etme).

In [19]:
import io
df = pd.read_excel(io.BytesIO(uploaded['YetGen_Mock_Data.xlsx']))
df

,id,first_name,last_name,gender,Not 1,Not 2,Not 3
0,1,Denver,O'Cullinane,Male,24,40,56
1,2,Florette,Minillo,Female,80,21,14
2,3,Chan,Benne,Male,88,23,59
3,4,Justino,Mirfield,Male,84,37,54
4,5,Norma,Boulter,Female,2,50,6
...,...,...,...,...,...,...,...
995,996,Prent,Dering,Male,27,21,7
996,997,Rosalinda,Sacaze,Female,8,24,73
997,998,Ansell,Pantridge,Male,29,56,55
998,999,Anabella,Delgua,Female,79,100,84


**Açıklama**:

- Toplam Not Hesaplama: Her satır için Not 1, Not 2, ve Not 3 sütunlarını toplar ve toplam_not adlı yeni bir sütun ekler.

- Filtreleme: toplam_not değeri 100 veya daha az olan satırları seçer ve bu satırları filtered_df içinde saklar.

- df: Verilerin güncellenmiş hali, toplam not sütunu ile birlikte görüntülenir.

In [20]:
# Toplam not sütunu ekleme
df['toplam_not'] = df['Not 1'] + df['Not 2'] + df['Not 3']

# Toplam notu 100'den küçük olanları filtreleme
filtered_df = df[df['toplam_not'] <= 100]
df


,id,first_name,last_name,gender,Not 1,Not 2,Not 3,toplam_not
0,1,Denver,O'Cullinane,Male,24,40,56,120
1,2,Florette,Minillo,Female,80,21,14,115
2,3,Chan,Benne,Male,88,23,59,170
3,4,Justino,Mirfield,Male,84,37,54,175
4,5,Norma,Boulter,Female,2,50,6,58
...,...,...,...,...,...,...,...,...
995,996,Prent,Dering,Male,27,21,7,55
996,997,Rosalinda,Sacaze,Female,8,24,73,105
997,998,Ansell,Pantridge,Male,29,56,55,140
998,999,Anabella,Delgua,Female,79,100,84,263


**Açıklama**:

1. Yeni Sheet'e Yazma
- file_path = 'YetGen_Mock_Data.xlsx': Excel dosyasının yolunu ve adını tanımlar. Bu, mevcut dosyayı güncelleyeceğimiz dosya adıdır.
- pd.ExcelWriter(...): Excel dosyasına yazmak için kullanılan pandas fonksiyonudur.
- file_path: Yazılacak dosyanın adı ve yolu.
- engine='openpyxl': Openpyxl, Excel dosyalarını okuma ve yazma işlemlerini gerçekleştirmek için kullanılan bir kütüphanedir.
- mode='a': Dosyayı ekleme (append) modunda açar. Bu, mevcut dosyanın üzerine yazmadan, yeni verileri dosyaya eklemenizi sağlar.
- filtered_df: Daha önce filtrelenmiş verileri içeren DataFrame.
- sheet_name='Gecerli_Notlar': Yeni sheet'in adı. Filtrelenmiş veriler bu sheet'e yazılacak.
- index=False: DataFrame'in indeks sütununu Excel dosyasına yazmaz. Yalnızca veri sütunları yazılır.
2. Yeni Dosyayı İndirme
- files.download('YetGen_Mock_Data.xlsx'): Google Colab ortamında, belirtilen dosya adıyla (bu durumda YetGen_Mock_Data.xlsx) güncellenmiş dosyayı indirmesi için tarayıcıya bir indirme bağlantısı sunar. Böylece, kullanıcı güncellenmiş Excel dosyasını bilgisayarına indirebilir.
3. Güncellenmiş DataFrame'i Görüntüleme
- df: Güncellenmiş DataFrame'i görüntüler. Bu adım, toplam_not sütunu eklenmiş ve diğer işlemler yapılmış DataFrame'in ekran çıktısını gösterir.

In [21]:
# Yeni bir sheet'e yazma (openpyxl kullanarak)
file_path = 'YetGen_Mock_Data.xlsx'
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    filtered_df.to_excel(writer, sheet_name='Gecerli_Notlar', index=False)

# Yeni dosyayı indirme
files.download('YetGen_Mock_Data.xlsx')
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,id,first_name,last_name,gender,Not 1,Not 2,Not 3,toplam_not
0,1,Denver,O'Cullinane,Male,24,40,56,120
1,2,Florette,Minillo,Female,80,21,14,115
2,3,Chan,Benne,Male,88,23,59,170
3,4,Justino,Mirfield,Male,84,37,54,175
4,5,Norma,Boulter,Female,2,50,6,58
...,...,...,...,...,...,...,...,...
995,996,Prent,Dering,Male,27,21,7,55
996,997,Rosalinda,Sacaze,Female,8,24,73,105
997,998,Ansell,Pantridge,Male,29,56,55,140
998,999,Anabella,Delgua,Female,79,100,84,263
